In [1]:
import pandas as pd
import datetime as dt
from datetime import timedelta
import dateutil
from dateutil.parser import parse
import logging
import importlib
importlib.reload(logging)

<module 'logging' from '/usr/local/anaconda3/lib/python3.6/logging/__init__.py'>

In [2]:
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.DEBUG, datefmt='%I:%M:%S')

In [3]:
logging.info("now starting...")

11:58:45 INFO:now starting...


In [4]:
logging.info("...before reading csv file.")
# read a csv into a dataframe
df = pd.read_csv('data/IMPORT_CONTROL.txt')
logging.info("...after reading csv file.")

11:58:52 INFO:...before reading csv file.


FileNotFoundError: File b'data/IMPORT_CONTROL.txt' does not exist

In [ ]:
logging.info("...before creating new column.")
# need to make a field to uniquely identify each agency
# concatenating the SITE_ID and the AGENCY will do this
named = df.assign(uniquename = df.SITE_ID.astype(str) + '-' + df.AGENCY.astype(str))
logging.info("...after creating new column.")

# new dataframe to record new info
interval_df = pd.DataFrame(columns=['uniquename', 'timestamp', 'interval', 'origindex'])


In [ ]:
# use the groupby functions to look at events for each agency
logging.info("...before groupby.")
grouped = named.groupby('uniquename')
logging.info("...after groupby.")

In [ ]:
logging.info("...before dict")
# make a dictionary of the group names
groups = dict(list(grouped))
logging.info("...after dict")


In [ ]:
i = 0
# for each group
for name in groups:
    # print(name)
    # get the group as a dataframe
    logging.info("...before get_group({})".format(name))
    g = grouped.get_group(name)
    logging.info("...after get_group({})".format(name))
    logging.info("length of group: {}".format(len(g)))
    j = 0
    # then go through the dataframe row by row
    for r in g.itertuples():
        logging.info("..on row# {}".format(r))
        # only for rows > 0
        if j > 0 and isinstance(r.PROCESS_END_TS, basestring):
            # calculate seconds between this timestamp and the timestamp from the last row
            thistime = parse(r.PROCESS_END_TS)
            lasttime = parse(last)
            secs = (thistime - lasttime).total_seconds()
            # print(r.Index, thistime, lasttime, secs)
            interval_df = interval_df.append({'timestamp': thistime, 'uniquename': name, 'interval': secs, 'origindex': r.Index}, ignore_index=True)
            last = r.PROCESS_END_TS
        if j == 0 and isinstance(r.PROCESS_END_TS, basestring):
            last = r.PROCESS_END_TS
        j += 1
    i += 1
    if i > 200000:
        break
    if i == 1 or i%50 == 0:
        logging.info("agency #{} out of {}".format(i, len(groups)))
        

interval_df.to_csv('intervals.csv')